<br><br>
# Explainable k-means

In [2]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

<br><br>
## Define funcs
I did not create Class of their algorithm, so this is a little experiment for me.

In [143]:
#ソートと動的計画法によって，最適な分割を求める (k=2)
#this func is main func and get best split condition using sort and dynamic programming
def optimal_threshold_2means(X):
    bests_split = {'cost':np.inf,'coordinate':None,'threshold':None}
    data_num = X.shape[0]
    data_dimentions = X.shape[1]
    u = np.sum(X*X)
    
    for i in range(data_dimentions):
        s = np.zeros(data_dimentions)
        r = np.sum(X,axis=0)
        ith_sorted_X = X[X[:,i].argsort(), :]
        for j,data in enumerate(ith_sorted_X[:-1]):
            s += data
            r -= data
            cost = u - np.sum(s*s)/(j+1) -np.sum(r*r)/(data_num-j-1)
            #print(cost)
            if cost < bests_split['cost'] and X[j][i]!=X[j+1][i]:
                bests_split['cost'] = cost
                bests_split['coordinate'] = i
                bests_split['threshold'] = data[i]
            
    return bests_split

#最適な分割に基づいてクラスタリング(k=2)
#this func is for clustering datasets based on best splits got from above func
def clustering_2means_by_tree(bests_split,X):
    cluster = np.ones(X.shape[0])
    for i,data in enumerate(X):
        if(data[bests_split['coordinate']]>bests_split['threshold']):
            cluster[i] = 0
    return cluster

#得られた分割の中心座標を求める(k=2)
#this func is for calculating center points
def get_mean(X,approx_labels):
    res=[]
    for k in range(len(np.unique(approx_labels))):
        n = 0
        mean = np.zeros(X.shape[1])
        for i,data in enumerate(X):
            if(approx_labels[i]==k):
                mean+=data
                n+=1
        res.append(mean/n)
    return np.array(res)

#近似の比率を計算する，論文ではkmeansの場合，上界は4である(k=2)．
#this func is for calculating approximation ratio
def approx_score(approx_labels,kmeans_model,X):
    kmeans_cost = 0
    kmeans_label = kmeans_model.labels_
    kmeans_centers = kmeans_model.cluster_centers_
    for i,data in enumerate(X):
        kmeans_cost += np.sum((data-kmeans_centers[kmeans_label[i]])*(data-kmeans_centers[kmeans_label[i]]))
    
    approx_cost = 0
    mean = get_mean(X,approx_labels)
    for k in range(kmeans_model.n_clusters):
        for i,data in enumerate(X):
            if(approx_labels[i]==k):
                approx_cost += np.sum((data-mean[k])*(data-mean[k]))
    print(kmeans_cost)
    print(approx_cost)
    return approx_cost/kmeans_cost

<br><br>
## Two Datasets
First is from [this good article](https://qiita.com/ynakayama/items/1223b6844a1a044e2e3b).

Second is from uci repo and I get the code from [this good article](https://pythondatascience.plavox.info/scikit-learn/%E3%82%AF%E3%83%A9%E3%82%B9%E3%82%BF%E5%88%86%E6%9E%90-k-means).

In [144]:
# First dataset
# 生徒の国語・数学・英語の各得点を配列として与える
X = np.array([
        [  80,  85, 100 ],
        [  96, 100, 100 ],
        [  54,  83,  98 ],
        [  80,  98,  98 ],
        [  90,  92,  91 ],
        [  84,  78,  82 ],
        [  79, 100,  96 ],
        [  88,  92,  92 ],
        [  98,  73,  72 ],
        [  75,  84,  85 ],
        [  92, 100,  96 ],
        [  96,  92,  90 ],
        [  99,  76,  91 ],
        [  75,  82,  88 ],
        [  90,  94,  94 ],
        [  54,  84,  87 ],
        [  92,  89,  62 ],
        [  88,  94,  97 ],
        [  42,  99,  80 ],
        [  70,  98,  70 ],
        [  94,  78,  83 ],
        [  52,  73,  87 ],
        [  94,  88,  72 ],
        [  70,  73,  80 ],
        [  95,  84,  90 ],
        [  95,  88,  84 ],
        [  75,  97,  89 ],
        [  49,  81,  86 ],
        [  83,  72,  80 ],
        [  75,  73,  88 ],
        [  79,  82,  76 ],
        [ 100,  77,  89 ],
        [  88,  63,  79 ],
        [ 100,  50,  86 ],
        [  55,  96,  84 ],
        [  92,  74,  77 ],
        [  97,  50,  73 ],
        ])

In [145]:
#second datasets
cust_df = pd.read_csv("http://pythondatascience.plavox.info/wp-content/uploads/2016/05/Wholesale_customers_data.csv")
del(cust_df['Channel'])
del(cust_df['Region'])
cust_array = np.array([cust_df['Fresh'].tolist(),
                       cust_df['Milk'].tolist(),
                       cust_df['Grocery'].tolist(),
                       cust_df['Frozen'].tolist(),
                       cust_df['Milk'].tolist(),
                       cust_df['Detergents_Paper'].tolist(),
                       cust_df['Delicassen'].tolist()
                       ], np.int32)
cust_array = cust_array.T
cust_array.shape

(440, 7)

<br><br>
## First data and k=2 

In [154]:
#K-meansクラスタリングをおこなう
kmeans_model = KMeans(n_clusters=2, random_state=10).fit(X)
#分類先となったラベルを取得する
labels = kmeans_model.labels_

#提案手法による近似アルゴリズムで取得する
bests_split = optimal_threshold_2means(X)
approx_labels = clustering_2means_by_tree(bests_split,X)
print(bests_split)
print(approx_labels)

{'cost': 11316.030172413797, 'coordinate': 0, 'threshold': 70}
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [147]:
approx_score(approx_labels,kmeans_model,X) #近似アルゴリズムのコスト ➗ k-meansのコスト

11316.030172413793
11316.030172413793


1.0

簡単だったためか,0番目の特徴量を閾値70で分割することで，k-meansと同じクラスタリングを実現できている

<br><br>
## Second data and k=2

In [148]:
#K-meansクラスタリングをおこなう
kmeans_model = KMeans(n_clusters=2, random_state=10).fit(cust_array)
#分類先となったラベルを取得する
labels = kmeans_model.labels_

#提案手法による近似アルゴリズムで取得する
bests_split = optimal_threshold_2means(cust_array)
approx_labels = clustering_2means_by_tree(bests_split,cust_array)
print(bests_split)

{'cost': 41288343652.5358, 'coordinate': 2, 'threshold': 16483}


In [149]:
approx_score(approx_labels,kmeans_model,cust_array) #近似アルゴリズムのコスト ➗ k-meansのコスト

132340344661.33641
135777624164.53575


1.0259730281948067

<br><br>
## IMM Algorithm
I'm working on this....

## Define funcs
I did not create Class of their algorithm, so this is a little experiment for me.

In [131]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None,condition=None):
        self.val = val
        self.left = left
        self.right = right 
        self.condition = (0,0) #(i,threshold) x_i <= threshold  or  x_i > threshold 

def minimum_center(i,labels,centers):
    minimum = np.inf
    for j in labels:
        minimum = min(minimum, centers[j][i])
    return minimum

def maximum_center(i,labels,centers):
    maximum = np.inf
    for j in labels:
        maximum = min(maximum, centers[j][i])
    return maximum  

def mistake(x,center,i,threshold):
#     print(x)
#     print(center)
#     print(i)
#     print(threshold)
    return 0 if ((x[i]<=threshold) == (center[i]<=threshold)) else 1

def delete_mistakes_data(X,labels,centers,i,threshold):
    data = []
    for idx,x in enumerate(X):
        if(mistake(x,centers[labels[idx]],i,threshold)==0):
            data.append(x)
    return np.array(data)
    
def make_next_data(X,labels,i,threshold):
    l_data=[]
    l_labels=[]
    r_data=[]
    r_labels=[]
    for idx,x in enumerate(X):
        if(x[i]<=threshold):
            l_data.append(x)
            l_labels.append(labels[idx])
        else:
            r_data.append(x)
            r_labels.append(labels[idx])
            
    return np.array(l_data),np.array(l_labels),np.array(r_data),np.array(r_labels)

In [132]:
def count_mistakes(X,l,i,labels,centers):
    cnt=0
    for idx,x in enumerate(X):
        if(mistake(x,centers[labels[idx]],i,l[i])==1):
            cnt+=1
    return cnt

In [136]:
def get_best_splits(X,l,r,labels,centers):
    bests_split = {'mistake':np.inf,'coordinate':None,'threshold':None}
    data_num = X.shape[0]
    data_dimentions = X.shape[1]
    
    for i in range(data_dimentions):
        ith_sorted_X = X[X[:,i].argsort(), :]
        ith_sorted_centers = centers[centers[:,i].argsort(), :]
        idx_center = 1
        cnt_mistakes = count_mistakes(X,l,i,labels,centers)
        print(cnt_mistakes)
        for j,x in enumerate(X[:-1]):
            if(l[i]>x[i] or x[i]>=r[i]):
                continue
            cnt_mistakes -= 1
            if(x[i]>=ith_sorted_centers[idx_center][i]):
                cnt_mistakes = count_mistakes(X,x,i,labels,centers)
                idx_center+=1
                
            if bests_split['mistake'] > cnt_mistakes:
                bests_split['mistake'] = cnt_mistakes
                bests_split['coordinate'] = i
                bests_split['threshold'] = x[i]   
            
    return bests_split['coordinate'],bests_split['threshold']

In [139]:
def build_tree(X,labels,centers):
    node = TreeNode()
    l=[]
    r=[]
    
    #論文疑似コード ２〜４行目
    if(len(np.unique(labels))==1):
        node.val = labels[0]
        return node
    

    #論文疑似コード ６〜９行目
    for i in range(X.shape[1]):
        l.append(minimum_center(i,labels,centers))
        r.append(maximum_center(i,labels,centers))
    print(l)  
    #論文疑似コード 10〜13行目
    i,threshold = get_best_splits(X,l,r,labels,centers)
    print(i,threshold)
    X = delete_mistakes_data(X,labels,centers,i,threshold)
    left_data,left_labels,right_data,right_labels = make_next_data(X,labels,i,threshold)
    
    #論文疑似コード １４〜１６行目
    node.condition = (i,threshold)
    print(l)
    print(node.condition)
    print(left_data)
    node.left = build_tree(left_data,left_labels,centers)
    node.right = build_tree(right_data,right_labels,centers)
    
    return node

<br><br>
## First data and k=3

In [142]:
labels

array([1, 1, 0, 1, 1, 2, 1, 1, 2, 0, 1, 1, 2, 0, 1, 0, 2, 1, 0, 0, 2, 0,
       2, 0, 1, 1, 1, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2], dtype=int32)

In [140]:
#IMM procedure
kmeans_model = KMeans(n_clusters=3, random_state=10).fit(X)
centers = kmeans_model.cluster_centers_
labels = kmeans_model.labels_

root = build_tree(X,labels,centers)
root

[61.0, 73.07692307692308, 78.61538461538461]
5
11
8
None None


TypeError: '<=' not supported between instances of 'int' and 'NoneType'

<br><br>
## Second data and k=3

In [89]:
#IMM procedure
kmeans_model = KMeans(n_clusters=3, random_state=10).fit(X)
centers = kmeans_model.cluster_centers_
labels = kmeans_model.labels_

root = build_tree(X,labels,centers)
root

[]


IndexError: tuple index out of range